In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 LangChain 在 Vertex AI 上進行 SQL 程式碼產生 🦜🔗

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation_langchain.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
| 作者: | [Shubham Chawla](https://www.github.com/shubhamgoogle), [Roy Arsan](https://www.linkedin.com/in/arsan) |


## 概述
大語言模型可於產生程式碼，包含 SQL。特別是，模型可以將自然語言文字轉換成 SQL 查詢。其中一個常見的目的是讓使用者在不需要知道表格名稱、資料架構及基礎資料倉儲中的特定 SQL 方言或查詢引擎 (例如 BigQuery) 的情況下，也可以查詢資料。

本記事涵蓋 SQL 程式碼產生中提示工程的最佳實務，以及透過 [Langchain Google Cloud Vertex AI](https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm) 執行，並於 [SQL-PaLM: 改善大型語言模型適應，以用於文字轉 SQL](https://arxiv.org/pdf/2306.00739.pdf) 中實行所學到的知識。例如，BigQuery 資料集架構會以動態方式擷取，並作為提示的內容提供，以奠定 LLM 的基礎並個人化其輸出。本記事也示範檢索加強產生 (RAG)，方法是使用來自 Langchain 範例選擇器的 [SemanticSimilarityExampleSelector](https://python.langchain.com/docs/modules/model_io/prompts/example_selector_types/similarity) 來動態擷取並傳遞最相關的幾種範例到豐富的 LLM 提示。這有助於確保最準確、最相關的 LLM 輸出，也就是產生的 SQL 查詢，同時限制所需的 LLM 輸入 Token 數量，進而降低成本。本記事也示範簡單的模型評估，其中產生的 SQL 查詢透過針對 BigQuery 資料集執行來評估，並透過將其與真實查詢及相應結果進行比較。

在本記事中，你會產生 SQL 查詢來分析 Cloud Audit Logs，並回答你自己的 Google Cloud 專案中的活動周遭關鍵安全性問題。雖然本文使用 BigQuery 日誌資料集，但本文提出的概念和方法也可以套用於其他資料庫與資料集。

![NL2SQL flow](https://services.google.com/fh/files/misc/nl2sql_for_log_analytics2.png)


### 目標

在完成筆記本後，你將可以：

* 使用模型根據自然語言問題產生 SQL 查詢：
  * 使用 VertexAIEmbeddings 建立內嵌
  * 使用 Langchain 範例選擇器自動為小量示範提示選擇相關範例
  * 提供自訂資料集架構作為內容
  * 格式化模型輸出

* 評估模型產生之查詢，方法為：
  * 對實際資料集執行已調整的查詢
  * 使用 pandas 資料框相等檢查，將查詢 (及其結果) 與 ground truth 查詢進行比較
  * 計算模型準確度分數

此外，你可以使用這個筆記本從你自己的稽核記錄中回答你自己的安全性問題，例如：

- 過去一個月內，是否有不正常的高雲端 API 使用量由任何使用者身分使用？
- 過去 7 天內，是否有不適當的身分執行任何破壞性行為？
- 本週內，是否有使用者存取到異常大量資料，導致每日資料量暴增？


## 開始使用


### 必備條件
如果你尚未執行此操作，唯一的需求是將你現有的記錄區塊 [升級](https://cloud.google.com/logging/docs/buckets#upgrade-bucket) 至 Log Analytics，這樣便能為你連結一個 BigQuery 資料集，裡面有可進行查詢的記錄資料。這是 **一鍵完成步驟，不會產生額外的費用** 。預設情況下，每個專案的 _Required_ 區塊會啟用、擷取並儲存 Cloud Audit 管理員活動記錄，不會產生任何費用。

![one click prerequisite](https://services.google.com/fh/files/misc/upgrade_log_bucket.png)


### 安裝 SDK


In [ ]:
# Install Vertex AI SDK to use for model predictions
!pip install google-cloud-aiplatform google-cloud-bigquery --upgrade --user
!pip install langchain
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai
!pip install --upgrade --quiet langchain langchain-google-vertexai
!pip install faiss-cpu

**僅限 Colab：** 取消以下Cell註解以重新啟動核心或使用按鈕重新啟動核心。對於 Vertex AI Workbench，你可以使用右上方的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython
# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 引進函式庫


In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS

import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import aiplatform
import sys

### 為 BigQuery 設定專案和資料集


這是包含下列項目的專案：
- 連結的 BigQuery 資料集 `BQ_LINKED_DATASET` (含你的原始記錄)，以及，
- 新的 BigQuery 資料集 `BQ_PROCESSED_DATASET`，用來儲存已處理的記錄。

這個專案可以是你用於 Vertex AI 的專案，也可以是不同的專案。

請確定你有 `BQ_LINKED_DATASET` 資料集的 **BigQuery Data Viewer** 角色


In [ ]:
PROJECT_ID = ""  # @param {type:"string"}
LOCATION_US = "US"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string
BQ_LINKED_DATASET = ""  # @param {type:"string"}
BQ_PROCESSED_DATASET = ""  # @param {type:"string"}

aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 驗證你的筆記本電腦環境
* 如果你使用 **Colab** 執行此筆記本電腦，執行下列Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看 [這裡](https://github.com/doggy8088/generative-ai/tree/main/setup-env) 的設定說明。


In [ ]:
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 設定嵌入和 Vertex AI LLM 模型

在目前的範例，我們正在使用 text-bison@002 大型語言模型，但你可以使用 Google 提供的其他模型 gemini、gemini-pro、ulta 等。對於嵌入，我們使用最新版本的 textembedding-gecko。


In [ ]:
MODEL_ID = "text-bison@002"  # @param {type:"string"}
EMBEDDING_MODEL_ID = "textembedding-gecko@latest"  # @param {type:"string"}

### 匯入樣本查詢


你現在會從CSV文件裡擷取15個範例安全問題和相應的SQL查詢清單。這些安全問題是變動自開原始碼[社群安全分析](https://github.com/GoogleCloudPlatform/security-analytics)的。CSA提供一組安全問題以及對應的查詢，適用於BigQuery、Log Analytics和Chronicle。

我們會使用這些查詢的子集作為模型提示範例中的小樣本，以及將剩餘的部分用於模型評估。

執行下列命令，從GCS儲存區讀取CSV文件，並載入所有記錄至記憶中的pandas DataFrame中：


In [ ]:
BUCKET_ID = "csa-datasets-public"  # @param {type:"string"}
FILENAME = "SQL_Generator_Example_Queries.csv"  # @param {type:"string"}
df = pd.read_csv(f"gs://{BUCKET_ID}/{FILENAME}", header=0)

### 萃取訓練 & 評估資料集


提取訓練和評估資料集，並儲存在各自的資料架構中：


In [ ]:
train_df = df.loc[df["Dataset"] == "Train", ["Question", "SQL Query"]]
eval_df = df.loc[df["Dataset"] == "Eval", ["Question", "SQL Query"]]
train_dict = (
    train_df[["Question", "SQL Query"]]
    .rename(columns={"SQL Query": "answer"})
    .rename(columns={"Question": "question"})
    .to_dict(orient="records")
)
train_df.head(2)

## 準備資料


> 如果你已使用 [Dataform 作為 Community Security Analytics 的一部分](https://github.com/GoogleCloudPlatform/security-analytics/tree/main/dataform) (CSA) 來處理原始記錄並使用經過整理的表格進行標準化，可以略過此部分。如需瞭解 CSA 以及如何自動持續建立經過後製處理的資料表以取代原始記錄，請查看 [Google Cloud 部落格文章](https://cloud.google.com/blog/products/data-analytics/deploy-community-security-analytics-with-dataform)。


如同任何其他的人工智慧 /機器學習專案，第一件事便是準備好資料，包括給少次提示和後續評估所用的資料集。你會將儲存在 BigQuery 連結資料集的原始記錄預先處理成摘要表格，並放到你的新 BigQuery 資料集裡。這個表格將以彙總方式顯示記錄，並正規化成一個簡單的架構。這讓你可以解鎖和縮放機器學習分析：
- 從運算觀點而言，因為這個資料集較小且簡單。
- 從人才觀點而言，因為研究人員和分析師不一定要熟悉原始記錄的複雜架構 ( [Log Entry 定義](https://cloud.google.com/logging/docs/reference/v2/rest/v2/LogEntry))。


### 建立新資料集


In [ ]:
!bq --location=LOCATION_US mk --dataset {BQ_PROJECT_ID}:{BQ_PROCESSED_DATASET}

使用記錄分析 Bigquery 表格建立新的 csa_4_01_summary_daily


In [ ]:
TABLE_NAME = "csa_4_01_summary_daily"
TABLE_ID = f"{PROJECT_ID}.{BQ_PROCESSED_DATASET}.{TABLE_NAME}"
SUMMARY_LOOKBACK_DAYS = 90

client = bigquery.Client(project=PROJECT_ID, location=LOCATION_US)
client.create_dataset(dataset=BQ_PROCESSED_DATASET, exists_ok=True)

job_config = bigquery.QueryJobConfig(
    destination=TABLE_ID, write_disposition="WRITE_TRUNCATE"
)

sql = f"""
SELECT
  EXTRACT(DATE FROM timestamp) AS day,
  proto_payload.audit_log.authentication_info.principal_email,
  ARRAY_AGG(DISTINCT proto_payload.audit_log.method_name IGNORE NULLS) AS actions,
  COUNT(*) AS counter
FROM `{PROJECT_ID}.{BQ_LINKED_DATASET}._AllLogs`
WHERE
  timestamp >=  TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL {SUMMARY_LOOKBACK_DAYS} DAY)
  AND proto_payload.audit_log.authentication_info.principal_email IS NOT NULL
  AND proto_payload.audit_log.method_name NOT LIKE "storage.%.get"
  AND proto_payload.audit_log.method_name NOT LIKE "v1.compute.%.list"
  AND proto_payload.audit_log.method_name NOT LIKE "beta.compute.%.list"
GROUP BY
  day,
  proto_payload.audit_log.authentication_info.principal_email
"""

# Start the query and save results in new table
query_job = client.query(sql, job_config=job_config)
result = query_job.result()  # Wait for the job to complete.

print(f"{result.total_rows} user action records loaded to table {TABLE_ID}")

### 建構 schema 定義 (精簡版）


首先，我們需要建立簡潔的資料集架構定義。如同前面所述，我們會將它用作提示背景的一部分來解決結果。


從 BigQuery 資料集的 `INFORMATION_SCHEMA` 中擷取資料表和欄位定義。


In [ ]:
# Following SQL query will generate schema definition of your dataset

BQ_TABLES = df["Qualified table name"].replace("", np.nan).dropna().unique()
print(BQ_TABLES)
QUERY = f"""\
SELECT
    '[Schema (values)]: ' || '| log_summary | ' || STRING_AGG(table_values, ' | ') || ';' AS tables_definition,
    '[Column names (type)]: ' || STRING_AGG(column_names_types) || ';' AS columns_definition
FROM (
    SELECT
      table_name,
      table_name || ' : ' || STRING_AGG(column_name, ' , ') as table_values,
      STRING_AGG(table_name || ' : ' || column_name || ' (' || data_type || ')', ' | ') as column_names_types
    FROM {PROJECT_ID}.{BQ_PROCESSED_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    WHERE table_name IN {'(' + ",".join(map(lambda x: f"'{x}'", BQ_TABLES)) + ')'}
    GROUP BY table_name
    ORDER BY table_name
)
"""

# Create query job
query_job = client.query(QUERY)
# Get first row
schema = next(query_job.result())

# Build schema definition
schema_definition = f"""\
{schema.tables_definition}

{schema.columns_definition}
"""

print(schema_definition)

### 建立提示

我們使用使用者輸入和使用 Langchain FewShotPromptTemplate 動態提取的幾個鏡頭來建立提示


#### 建立 VertexAI embeddings 以建立文字表徵


In [ ]:
embedding = VertexAIEmbeddings(model_name=EMBEDDING_MODEL_ID, project=PROJECT_ID)

#### 建立少量提示

`SemanticSimilarityExampleSelector` 選擇範例的依據，為範例和輸入最相似的組合


In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    train_dict,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embedding,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # This is the number of examples to produce.
    k=2,
)

# Select the most similar example to the input.
question = "select user actions that contains the word 'delete' or 'remove'"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

#### 用於建立提示的 Helper 函式

下方函式將用於將使用者輸入轉換成實際提示，其中包含少量範例和開頭


In [ ]:
def build_prompt(user_prompt, example_selector):
    prompt_template = f"""\
    This is a task converting text into GoogleSQL statement.
    We will first give you the dataset schema and then ask a question in text.
    You are asked to generate SQL statement which is valid for BigQuery.
    Remove any delimiters around answer such as "```"

    BigQuery tables schema definition:
    {schema_definition}
    Here are a few shot examples:
    """
    example_prompt = PromptTemplate(
        input_variables=["question", "answer"],
        template="question: {question}\nanswer: {answer}",
    )

    prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix=prompt_template,
        suffix="question: {question}\nanswer: ",
        input_variables=["question"],
    )
    final_prompt = prompt.format(question=user_prompt)
    return final_prompt

## 生成 SQL 查詢


### 定義用於產生 SQL 的輔助函式


`generate_sql()`: 此函式用於從 Vertex AI LLM 模型中擷取 SQL 查詢，並使用我們迄今為止建置的提示範本。

`execute_sql()`: 此函式用於針對實際 BigQuery 資料集執行 SQL 查詢，並將結果傳回為資料框。

`build_prompt()`: 此函式用於建立最終提示，其中包含所有提示的共用字首和字尾

請注意，`generate_sql()` 如何使用 `sanitize_output()` 函式在回傳結果前將回應分解為 SQL 查詢本身。即使模型提示包含調整模型輸出的指示，仍然可能會有需要移除的引號或程式區塊反引號，以避免後續的 SQL 語法錯誤。


In [ ]:
# Limit number of bytes processed as a guardrail for cost control
BQ_MAX_BYTES_BILLED = pow(2, 30)  # 1GB


def execute_sql(query: str):
    print("Executing SQL...")

    # Qualify table names with your project and dataset ID
    for table_name in BQ_TABLES:
        query = query.replace(
            table_name, f"{PROJECT_ID}.{BQ_PROCESSED_DATASET}.{table_name}"
        )

    # Validate the query by performing a dry run without incurring a charge
    job_config = bigquery.QueryJobConfig(use_query_cache=False, dry_run=True)
    try:
        response = client.query(query, job_config=job_config)
    except Exception as e:
        print("Error validating query:")
        print(e)
        return e

    print("Query will process {:.2f} KB.".format(response.total_bytes_processed / 1024))

    # Execute the query
    job_config = bigquery.QueryJobConfig(
        use_query_cache=False, maximum_bytes_billed=BQ_MAX_BYTES_BILLED
    )
    try:
        response = client.query(query)
        df = response.to_dataframe()
    except Exception as e:
        print("Error executing query:")
        print(e)
        return e

    return df


import re


# Strip text to include only the SQL code block with
def sanitize_output(text: str) -> str:
    # Strip whitespace and any potential backticks enclosing the code block
    text = text.strip()
    regex = re.compile(r"^\s*```(\w+)?|```\s*$")
    text = regex.sub("", text).strip()

    # Find and remove any trailing quote without corresponding opening quote
    if re.search(r'^[^"]*"$', text):
        text = text[:-1]
    # Find and remove any leading quote without corresponding closing quote
    if re.search(r'^"[^"]*$', text):
        text = text[1:]

    return text


# Call model using prompt and pre-defined parameters
def generate_sql(
    prompt: str,
    temperature: float = 0.2,
    max_output_tokens: int = 1024,
    top_k: int = 40,
    top_p: float = 0.8,
) -> str:
    print("Generating SQL...")
    print("Number of input tokens: " + str(len(prompt)))

    model = VertexAI(
        model_name=MODEL_ID,
        temperatore=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,
    )
    final_prompt = build_prompt(prompt, example_selector)
    print(final_prompt)
    text = model.invoke(final_prompt)
    print("Number of output tokens: " + str(len(text)))
    print("Response:")
    print(text)
    # Strip text to include only the SQL code block
    text = sanitize_output(text)
    print("Response stripped:")
    print(text)

    return text

### 範例 1


讓我們生成 SQL 來回答這個範例問題：

*列出包含「刪除」或「移除」字詞的所有使用者操作，時段涵蓋上個月。在結果中包含使用者與日期。*


In [ ]:
user_prompt = "List all user actions that contains the word 'delete' or 'remove' over the last month. Include the user and the day in the results."

final_generated_prompt = build_prompt(user_prompt, example_selector)
print(final_generated_prompt)

我們使用 BigQuery 中的線上資料集測試所產生的查詢。


In [ ]:
output = generate_sql(example_selector, user_prompt)
print(output)

讓我們針對你的 BigQuery 資料集測試產生的查詢：


In [ ]:
# Execute the query
query_result = execute_sql(output)
display(query_result.head(2))

### 範例 2


讓我們生成 SQL 來回答這個範例問題：

*列出包含「刪除」或「移除」字詞的所有使用者操作，時段涵蓋上個月。在結果中包含使用者與日期。*


In [ ]:
user_prompt = "List any action containing IAM case-insensitive by any unapproved user over the last 7 days, where approved user include 'admin@example.com'"

final_generated_prompt = build_prompt(user_prompt, example_selector)
print(final_generated_prompt)

我們使用 BigQuery 中的線上資料集測試所產生的查詢。


In [ ]:
output = generate_sql(example_selector, user_prompt)
print(output)

讓我們針對你的 BigQuery 資料集測試產生的查詢：


In [ ]:
# Execute the query
query_result = execute_sql(output)
display(query_result.head(2))

## 評估模型


### 在評估資料集上運行模型


讓我們為評估資料集中所有的問題產生 SQL 查詢。該資料集同時包括「問題」和正解「SQL 查詢」。執行以下程式碼，為資料集中的每個問題自動呼叫模型，並將回應記錄在新的欄位「已產生的 SQL 查詢」。由於模型呼叫是串列執行的，這可能需要幾分鐘。


In [ ]:
eval_df["Generated SQL Query"] = eval_df["Question"].apply(
    lambda x: generate_sql(example_selector, x)
)

### 比較輸出結果

在下一單元格中，我們將執行原始的 SQL 查詢，然後直接比對它的輸出與所產生 SQL 查詢的輸出。


In [ ]:
def compare_dataframes(sql_query, generated_sql_query):
    """Compares two pandas DataFrames row-wise using columns from the second DataFrame.
    Args:
        SQL Query, Generated SQL Query
    Returns:
        True if output of both the SQL queries matches otherwise False
    """
    df1 = execute_sql(sql_query)
    df2 = execute_sql(generated_sql_query)

    # If generated query returned an error instead of a dataframe with results:
    if not isinstance(df2, pd.DataFrame):
        return False

    try:
        df2 = df2[df1.columns]
    except KeyError:
        # Columns in results of ground truth query are missing
        # from results returned by generated query
        return False

    comparison_result = df2.eq(df1)
    matching_rows = comparison_result.all(axis=1)
    matching_count = matching_rows.sum()
    # return df1, df2
    return True if matching_count == len(df1) else False


eval_df["Data Match"] = eval_df.apply(
    lambda x: compare_dataframes(x["SQL Query"], x["Generated SQL Query"]), axis=1
)
# eval_df["sql_query_output"],eval_df["generated_sql_query_output"] = eval_df.apply(lambda x: compare_dataframes(x["SQL Query"], x["Generated SQL Query"]), axis=1)

# Note: To save the output data to the final dataframe, make these changes: 1. Uncomment lines 26 and 30. 2. Comment out line 29.

## 最終結果

在以下單元格中，我們將計算模型的最終分數。這個分數表示原始查詢和生成查詢之間成功匹配的百分比，如「資料配對」欄中所示。


### 分數


In [ ]:
def get_prcntg_match(eval_df):
    return round(eval_df["Data Match"].sum() / len(eval_df) * 100)


prcntg_match = get_prcntg_match(eval_df)
print(f"Final Score based on the percentage of data match: {prcntg_match}%")

### 輸出


In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", -1)
display(eval_df)